In [ ]:
from scripts.experiments.symmetries import pauli_reduce, clifford_symmetry, get_coupled_qudits_by_gate
from sympleq.models.Ising import ising_chain_hamiltonian, ising_2d_hamiltonian
from sympleq.core.circuits.gate_decomposition_to_circuit import gate_to_circuit
import numpy as np

In [ ]:

N = 9
J = 1
h = 0.5
H = ising_chain_hamiltonian(N, J, h, periodic=True)

# print(H)

h_red, conditioned_hamiltonians, C_F, all_phases = pauli_reduce(H)
# print(h_red)
# print(conditioned_hamiltonians)


In [ ]:
H_c = H  # conditioned_hamiltonians[0]
F, S, T = clifford_symmetry(H_c, check_symmetry=True)

C_F = gate_to_circuit(F)
# C_S = gate_to_circuit(S)
# C_T = gate_to_circuit(T)
assert C_F.act(H_c).to_standard_form() == H_c.to_standard_form()
# assert C_S.act(C_T.act(H_c)).standard_form() == H_c.standard_form()
U_F = C_F.unitary().toarray()
H_hilbert = H_c.to_hilbert_space().toarray()
assert np.all(np.abs(U_F @ H_hilbert @ U_F.conj().T - H_hilbert) < 1e-8)
print('Found unitary symmetry!!!')

In [ ]:
# print(S.symplectic)
# assert T.inv().act(S.act(T.act(H_c))).standard_form() == H_c.standard_form()
print(H_c.n_qudits())
print(get_coupled_qudits_by_gate(F))
print(get_coupled_qudits_by_gate(S))

In [ ]:
n_x = 3
n_y = 2
J_zz = 1
h_x = 0.5
H_2d = ising_2d_hamiltonian(n_x, n_y, J_zz, h_x, periodic=False)

h_red, conditioned_hamiltonians, C_F, all_phases = pauli_reduce(H_2d)
print(conditioned_hamiltonians[0].n_qudits())
print(conditioned_hamiltonians[0].n_paulis())

In [ ]:
F, S, T = clifford_symmetry(conditioned_hamiltonians[0])


In [ ]:
print(F.symplectic)